In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def preprocess_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error: Unable to load image file '{image_path}'.")
        return None
    image = cv2.equalizeHist(image)
    image = cv2.resize(image, target_size)
    return image

In [ ]:
def apply_gabor_filters(image):
    kernels = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                kernel = cv2.getGaborKernel((21, 21), sigma, theta, frequency, 0.5, 0, ktype=cv2.CV_32F)
                filtered_image = cv2.filter2D(image, cv2.CV_32F, kernel)
                kernels.append(filtered_image)
    return np.array(kernels)

In [ ]:
def build_cnn(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def load_dataset(dataset_path, target_size=(128, 128)):
    images = []
    labels = []
    label_map = {}
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                image_path = os.path.join(root, file)
                label_folder = os.path.basename(root)
                if label_folder not in label_map:
                    label_map[label_folder] = len(label_map)
                label = label_map[label_folder]
                image = preprocess_image(image_path, target_size)
                if image is not None:
                    images.append(image)
                    labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
dataset_path = "/content/drive/MyDrive/dataset_FVC2000_DB4_B/dataset"
images, labels = load_dataset(dataset_path)

In [ ]:
gabor_features = []
for img in images:
    filtered_images = apply_gabor_filters(img)
    gabor_features.append(filtered_images)

gabor_features = np.array(gabor_features)

In [ ]:
X = gabor_features.reshape(len(gabor_features), gabor_features.shape[2], gabor_features.shape[3], gabor_features.shape[1])
y = labels

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the model
cnn_model = build_cnn((X_train.shape[1], X_train.shape[2], X_train.shape[3]))
cnn_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
cnn_model.evaluate(X_test, y_test)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8257 - loss: 5167.8228 - val_accuracy: 0.9818 - val_loss: 5339.1909
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9430 - loss: 5054.5225 - val_accuracy: 0.9818 - val_loss: 2880.7478
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9933 - loss: 1126.1949 - val_accuracy: 0.9818 - val_loss: 792.3975
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.9518 - loss: 628.2405 - val_accuracy: 0.9818 - val_loss: 882.9822
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.9839 - loss: 647.7166 - val_accuracy: 0.5091 - val_loss: 1002.1494
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9032 - loss: 335.2581 - val_accuracy: 0.9818 - val_loss: 544.5643
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.9691 - loss: 320.3873 - val_accuracy: 0.9818 - val_loss: 640.6447
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9884 - loss: 361.206

[25.261117935180664, 0.9818181991577148]